## 앙상블 학습

- 앙상블 기법: 여러 개의 분류기(classifier)를 생성하고 그 예측을 결합함으로써 보다 정확한 최종 예측을 도출하는 기법

- 대부분의 정형데이터 분류 시에는 앙상블이 뛰어난 성능을 나타내고 있음

- 대표적으로 랜덤포레스트와 그라디언트 부스팅 알고리즘이 있음

- 최신 기법으로는 XGBoost(eXtra Gradiant Boost)와 LightGBM(Light Gradiant Boost), Stacking이 주목 받음

#### <학습의 유형>

1. 보팅(Voting): 일반적으로 서로 다른 알고리즘을 가진 분류기를 결합하는 것

2. 배깅(Bagging): 각각의 분류기가 모두 같은 유형의 알고리즘 기반이지만, 데이터 샘플링을 서로 다르게 가져가면서 학습을 수행하는 것 >> 랜덤 포레스트

- 붓스트래핑(bootstrapping): 개별 classifier에게 데이터를 샘플링해서 추출하는 방식

- 교차 검증이 데이터셋 간에 중첩을 허용하지 않는 것과 달리 배깅은 중첩을 허용

3. 부스팅(Boosting): 여러 개의 분류기가 순차적으로 학습을 수행하되, 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해서는 올바르게 예측할 수 있도록 다음 분류기에 가중치를 부여하는 것

- 예측 성능이 뛰어나서 앙상블 기법을 주도하고 있음 >> 그라디언트 부스팅, XGBoost, LightGBM


+)스태킹(stacking): 여러가지 다른 모델의 예측 결과값을 다시 학습데이터로 만들어서 다른 모델(메타 모델)로 재학습시키는 것

### 보팅 유형

1. 하드 보팅(Hard Voting): 다수결 원칙과 비슷, 다수의 분류기가 결정한 예측값을 최종 보팅 결과값으로 선정하는 것

2. 소프트 보팅(Soft Voting): 분류기들의 레이블값 결정 확률을 모두 더하고 평균내서 확률이 가장 높은 레이블값을 최종 보팅 결과값으로 선정하는 것 >> 일반적으로 많이 사용

### 보팅 분류기(Voting Classifier)

- 사이킷런은 보팅 방식의 앙상블을 구현한 VotingClassifier 클래스 제공

#### 위스콘신 유방암 데이터세트를 예측 분석해보자

- 유방암의 악성종양, 양성종양 여부를 결정하는 이진 분류 데이터셋이며 종양의 크기, 모양 등의 형태와 관련한 피처를 가지고 있음

- load_breast_cancer()를 통해 데이터 로드 가능

In [1]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

date_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
date_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


#### 로지스틱 회귀, KNN(K 최근접 이웃) 기반의 소프트 보팅 방식

- VotingClassifier 클래스는 주요 인자로 estimators와 voting값을 입력받음

- estimators >> 여러 개의 튜플 타입 classifier 객체들을 리스트 값으로 입력 받음

- voting은 'hard' 또는 'soft'로 적용하면됨 >> 디폴트는 'hard'

In [5]:
lr_clf = LogisticRegression(solver='liblinear')
knn_clf = KNeighborsClassifier(n_neighbors=8)

#앙상블 모델 만들기
vo_clf = VotingClassifier(estimators=[('LR', lr_clf), ('KNN', knn_clf)], voting='soft')

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=156)

#모델 학습, 예측, 평가
vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

#개별 모델의 학습, 예측, 평가
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
  classifier.fit(X_train, y_train)
  pred = classifier.predict(X_test)
  class_name = classifier.__class__.__name__
  print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))

Voting 분류기 정확도: 0.9561
LogisticRegression 정확도: 0.9474
KNeighborsClassifier 정확도: 0.9386


- 보팅으로 여러 개의 기반 분류기를 결합한다고 해서 무조건 기반 분류기보다 성능이 향상되는 것은 아니다

- 그럼에도 불구하고 앙상블 기법은 전반적으로 다른 단일 ML 알고리즘보다 뛰어난 성능을 가진다

> 앙상블 학습에서는 결정 트리 알고리즘의 단점을 수십, 수천 개의 매우 많은 분류기를 결합하여 다양한 상황을 학습하게 함으로써 극복하고 있다